In [ ]:
import pandas as pd

trx_df = pd.read_csv(
    "data/Transactions.csv",
    encoding="utf-8-sig",
    sep=",",
    usecols=["Produit", "Code ISIN", "Quantité", "Montant négocié"],
)
trx_df.rename(columns={"Produit": "Product", "Code ISIN": "ISIN", "Quantité": "Quantity", "Montant négocié": "Negociated amount"}, inplace=True)

trx_df.head()

In [ ]:
portfolio_df = pd.read_csv(
    "data/Portfolio.csv",
    encoding="utf-8-sig",
    sep=",",
    usecols=["Ticker/ISIN", "Clôture", "Devise", "Montant en EUR"],
)
portfolio_df["Devise"] = portfolio_df["Devise"].str.split(" ").str[1]
portfolio_df.rename(columns={"Ticker/ISIN": "ISIN", "Clôture": "Market price", "Devise": "Amount in Original Currency", "Montant en EUR": "Amount in EUR"}, inplace=True)

# NA are not values that we need, let's drop them
portfolio_df = portfolio_df.dropna()

# Convert to float
portfolio_df["Market price"] = portfolio_df["Market price"].str.replace(",", ".", regex=False)
portfolio_df["Market price"] = portfolio_df["Market price"].astype(float)
portfolio_df["Amount in EUR"] = portfolio_df["Amount in EUR"].str.replace(",", ".", regex=False)
portfolio_df["Amount in EUR"] = portfolio_df["Amount in EUR"].astype(float)
portfolio_df["Amount in Original Currency"] = portfolio_df["Amount in Original Currency"].astype(float)

portfolio_df.head()

In [ ]:
portfolio_df["fx_rate"] = portfolio_df["Amount in EUR"] / portfolio_df["Amount in Original Currency"]
portfolio_df.head()

In [ ]:
# We don't need these columns anymore as fx_rate as been calculated
portfolio_df.drop(columns=["Amount in Original Currency", "Amount in EUR"], inplace=True)


df = pd.merge(trx_df, portfolio_df, left_on="ISIN", right_on="ISIN", how="left")

df.head()

In [ ]:
lines = df.groupby(["Product", "ISIN"]).agg({
    "Quantity": "sum",
    "Negociated amount": "sum",
    "Market price": "first",
    "fx_rate": "first"
})

lines

In [ ]:
# Market Price is NaN when all positions have been sold.
# Then, negociated amount is the PnL for this asset.

# When Market Price is not NaN, we still have all/some positions for this asset.
# Need to compute the unrealized PnL based on the market price.
# Negociated amount represents what we paid for this asset (cost basis), and might include some realized PnL.

lines.fillna({"Market price": 0, "fx_rate": 0 }, inplace=True)
lines["Realized and/or Unrealized PnL"] = lines["Negociated amount"] + (lines["Quantity"] * lines["Market price"] * lines["fx_rate"])

lines

In [ ]:
# Don't have the same result as in Degiro UI because of rounding - less precision in the CSV file for some assets market price
lines["Realized and/or Unrealized PnL"].sum()